# Ablation study on Amex data

In [4]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
import os
import gc
import cupy
import yaml
from datetime import datetime
import sklearn
from tqdm.auto import tqdm

from src.lib.experimentation import load_amex_numpy_data, undo_min_max_corrupt_func, fit_model, cross_validate_experiment
import pandas as pd
import numpy as np 
import seaborn as sns
from src.lib.plotting import get_figsize, save_plot, update_plot_params


with open(os.path.join("..", "config.yaml")) as f:
    main_cfg = yaml.load(f, Loader=yaml.FullLoader)
    
with open(os.path.join("..", "src", "experiments", "configs", "experiment-config-alpha.yaml")) as f:
    amex_cfg = yaml.load(f, Loader=yaml.FullLoader)

In [2]:
X_train, y_train = load_amex_numpy_data(
    os.path.join(main_cfg['dataset_directory'], "derived", "processed-splits"),
    amex_cfg['amex_dataset']['fill'],
    lambda X, y: undo_min_max_corrupt_func(X, y, 42),
    11,
)
print(X_train.shape)

(458913, 13, 188)


In [3]:
NUM_VAL_ITEMS = int(X_train.shape[0] * 2 / 10)

train_loader = torch.utils.data.DataLoader(
    dataset=torch.utils.data.TensorDataset(
        torch.from_numpy(X_train[NUM_VAL_ITEMS:]).float(),
        torch.from_numpy(y_train[NUM_VAL_ITEMS:]).float(),
    ), 
    **amex_cfg['amex_dataset']['data_loader']
)
val_loader = torch.utils.data.DataLoader(
    dataset=torch.utils.data.TensorDataset(
        torch.from_numpy(X_train[:NUM_VAL_ITEMS]).float(),
        torch.from_numpy(y_train[:NUM_VAL_ITEMS]).float(),
    ), 
    **amex_cfg['amex_dataset']['data_loader']
)

In [ ]:
 early_stopper_init_fn = lambda : EarlyStopper(
        patience=exp_cfg['fit']['early_stopper_patience'],
        min_delta=exp_cfg['fit']['early_stopper_min_delta'],
    )

In [27]:
from src.models.adaptive_grunet import AdaptiveGRUNet
from src.preprocessing.normalizing_flows import EDAIN_Layer
from src.preprocessing.static_transformations import (
    StandardScalerTimeSeries,
    IgnoreTimeDecorator,
)
from src.lib.experimentation import fit_model, cross_validate_experiment, EarlyStopper

TRAIN_EDAIN_MODEL = True

def get_model_init(outlier=True, shift=True, scale=True, power=True):
    return lambda : AdaptiveGRUNet(
        adaptive_layer=EDAIN_Layer(
            input_dim=177,
            invert_bijector=False,
            outlier_removal_residual_connection=True,
            batch_aware=False,
            init_sigma=0.000001,
            outlier_removal_mode='exp',
            adaptive_shift=shift,
            adaptive_scale=scale,
            adaptive_outlier_removal=outlier,
            adaptive_power_transform=power,
        ),
        num_features=188,
        num_cat_columns=11,
        hidden_dim=128,
        layer_dim=2,
        embedding_dim=4,
        time_series_length=13,
        dim_first=False,
    )
preprocess_init_fn = lambda : IgnoreTimeDecorator(
    scaler=StandardScalerTimeSeries(time_series_length=1),
    time_series_length=13,
)

DEV = torch.device('cuda', 7)

optim_init_fn = lambda model : torch.optim.Adam(
    model.preprocess.get_optimizer_param_list(
        base_lr=1e-3,
        # For local-aware
        # scale_lr=0.0001,
        # shift_lr=0.01,
        # outlier_lr=10.0,
        # power_lr=0.1,
        # below for global-aware
        scale_lr=0.01,
        shift_lr=0.01,
        outlier_lr=100.0,
        power_lr=10.0,
        # # below for EDAIN-KL
        # scale_lr: 0.0001
        # shift_lr: 0.01
        # outlier_lr: 10.0
        # power_lr: 0.001
    ) + [
        {'params' : model.gru.parameters(), 'lr' : 1e-3},
        {'params' : model.feed_forward.parameters(), 'lr' : 1e-3},
        {'params' : model.emb_layers.parameters(), 'lr' : 1e-3 },
    ], lr=1e-3)
scheduler_init_fn = lambda optim : torch.optim.lr_scheduler.MultiStepLR(optim, milestones=[4, 7], gamma=0.1)

early_stopper_init_fn = lambda : EarlyStopper(
   patience=3,
   min_delta=0.0,
)

In [28]:
# cross_validate_model ...
# then make standard scaler preprocessing init fn...
def get_history(model_init_fn):
    history = cross_validate_experiment(
            model_init_fn=model_init_fn,
            preprocess_init_fn=preprocess_init_fn,
            optimizer_init_fn=optim_init_fn,
            scheduler_init_fn=scheduler_init_fn,
            early_stopper_init_fn=early_stopper_init_fn,
            loss_fn=F.binary_cross_entropy,
            X=X_train,
            y=y_train,
            num_epochs=30, #exp_cfg['fit']['num_epochs'],
            dataloader_kwargs=amex_cfg['amex_dataset']['data_loader'],
            num_folds=5, #args.num_cross_validation_folds,
            device=DEV,
            random_state=42, #args.random_state,
            num_categorical_features=11, #exp_cfg['amex_dataset']['num_categorical_features'],
        )
    return history

In [29]:
history_full = get_history(get_model_init(True, True, True, True))
save_path = os.path.join(main_cfg['experiment_directory'], "ablation-study-1111.npy")
np.save(save_path, history_full)

Starting model training [1 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [2 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [3 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [4 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [5 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

In [30]:
history_0110 = get_history(get_model_init(False, True, True, False))
save_path = os.path.join(main_cfg['experiment_directory'], "ablation-study-0110.npy")
np.save(save_path, history_0110)


Starting model training [1 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [2 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [3 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [4 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [5 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

In [31]:
history_0111 = get_history(get_model_init(False, True, True, True))
save_path = os.path.join(main_cfg['experiment_directory'], "ablation-study-0111.npy")
np.save(save_path, history_0111)


Starting model training [1 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [2 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [3 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [4 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [5 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

In [32]:
history_1110 = get_history(get_model_init(True, True, True, False))
save_path = os.path.join(main_cfg['experiment_directory'], "ablation-study-1110.npy")
np.save(save_path, history_1110)


Starting model training [1 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [2 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [3 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [4 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [5 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

In [33]:
history_0100 = get_history(get_model_init(False, True, False, False))
save_path = os.path.join(main_cfg['experiment_directory'], "ablation-study-0100.npy")
np.save(save_path, history_0100)


Starting model training [1 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [2 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [3 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [4 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [5 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

In [34]:
history_0010 = get_history(get_model_init(False, False, True, False))
save_path = os.path.join(main_cfg['experiment_directory'], "ablation-study-0010.npy")
np.save(save_path, history_0010)

Starting model training [1 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [2 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [3 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [4 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [5 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

In [35]:
history_0000 = get_history(get_model_init(False, False, False, False))
save_path = os.path.join(main_cfg['experiment_directory'], "ablation-study-0000.npy")
np.save(save_path, history_0000)


Starting model training [1 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [2 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [3 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [4 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

Starting model training [5 / 5]
Using device = cuda:7


  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
history_0111 = get_history(get_model_init(False, True, True, True))
history_1110 = get_history(get_model_init(True, True, True, False))

In [25]:
save_path = os.path.join(main_cfg['experiment_directory'], "ablation-study-test.npy")
np.save(save_path, history_full)

In [13]:
edain_model.preprocess.transform_list

[AdaptiveOutlierRemoval(),
 AdaptiveShift(),
 AdaptiveScale(),
 AdaptivePowerTransform()]

In [14]:
F.cross_entropy

<function torch.nn.functional.cross_entropy(input: torch.Tensor, target: torch.Tensor, weight: Optional[torch.Tensor] = None, size_average: Optional[bool] = None, ignore_index: int = -100, reduce: Optional[bool] = None, reduction: str = 'mean', label_smoothing: float = 0.0) -> torch.Tensor>

In [21]:
dev = torch.device('cuda', 4)

def evaluate_model(mod, outlier_removal=True, shift=True, scale=True, power_transform=True):
    running_vloss = 0.0
    mod = mod.to(dev)
    with torch.no_grad():
        for i, vdata in tqdm(enumerate(train_loader), desc="Evaluating model"):
            vinputs, vlabels = vdata
            vinputs, vlabels = vinputs.to(dev), vlabels.to(dev)
            
            # apply the preprocessing
            # ...
            if i == 0:
                mod.train()
            elif i == 1:
                mod.eval()
            voutputs = mod(vinputs)
            vloss = F.cross_entropy(voutputs, vlabels, reduction='mean').cpu().item()
            # vmetric = amex_metric_mod(vlabels.cpu().numpy(), voutputs.cpu().numpy())
            if i > 0:
                running_vloss += vloss
            # running_vmetric += vmetric
    avg_vloss = running_vloss / (i + 1)
    return avg_vloss


evaluate_model(edain_model, False, True, True, False)

Evaluating model: 0it [00:00, ?it/s]

1828.3663323258554